# Towards Visually Explaining Variational Autoencoders
To run this notebook, activate the conda environment via the environment.yml file and start jupyter notebook within this environmnent. 

Pretrained networks for the different models are available at:

https://drive.google.com/drive/folders/1OrX5HuH6vjSD8D2gLLv7ANc1iBUuVGHY?usp=sharing

In [ ]:
cd Anomaly_Detection

In [ ]:
import random

## MNIST Anomaly Detection

In [ ]:
run -t code/test_expVAE.py --dataset=mnist --model=vanilla_mnist --batch_size=32 --model_path='/media/bob/OS/Users/boble/Documents/AI - year 1/FACT-AI/MNIST_model/MNIST_model/7/model_best.pth'  --target_layer='encoder.2'

In [ ]:
run -t code/show_results.py --dataset=mnist --model=vanilla_mnist 

In [ ]:
img = random.choice([x for x in os.listdir("test_results/vanilla")
               if os.path.isfile(os.path.join("test_results/vanilla", x))], )
print(img)

## USCD-Ped1 Anomaly Detection

**Info:**  training one model can take up to 1 hour on a gpu. Downloading the dataset takes up 5 Gb.

### To train the authors configuration:

In [ ]:
run -t code/train_expVAE.py --model vanilla_ped1 --dataset ucsd_ped1 --batch_size 32 --epochs 512 --batch_norm False

### To test the authors configuration:
Make sure you download the model first and put it in the ```./ckpt``` folder.

In [ ]:
run -t code/test_expVAE.py --model vanilla_ped1 --dataset ucsd_ped1 --batch_size 4 --model_path './ckpt/vanilla_ped1_best_original.pth' --target_layer encoder.5 --image_size 100 --batch_size False

### To test the our best configuration:

In [ ]:
run -t code/test_expVAE.py --model vanilla_ped1 --dataset ucsd_ped1 --batch_size 4 --model_path './ckpt/vanilla_ped1_best_our_best.pth' --target_layer encoder.8 --image_size 96 --batch_norm True

### To compute the VAE reconstruction baseline

In [ ]:
run -t code/test_vanVAE.py --image_size 96 --reconstruction True --model vanilla_ped1 --dataset ucsd_ped1 --batch_size 4 --model_path './ckpt/vanilla_ped1_best_our_best.pth' --image_size 96

### To compute the new Average VAE reconstruction baseline

In [ ]:
run -t code/test_vanVAE.py --image_size 96 --reconstruction False --model vanilla_ped1 --dataset ucsd_ped1 --batch_size 4 --model_path './ckpt/vanilla_ped1_best_our_best.pth' --image_size 96

## MVTec Anomaly Detection

### To train the models:

**Info:**  training one model can take up to 1 hour on a gpu. Downloading the dataset takes up 5 Gb.

Specify the target as the index according to the following list(e.g. bottle: 0, cable: 1): 

['bottle', 'cable', 'capsule', 'carpet', 'grid',
               'hazelnut', 'leather', 'metal_nut', 'pill', 'screw',
               'tile', 'toothbrush', 'transistor', 'wood', 'zipper']

In [ ]:
object_class = 5 # 5 is nut
pre_trained_model= './ckpt/resnet18_3_mvtecClass_'+str(object_class) +'_final.pth'
self_trained_model = './ckpt/resnet18_3_mvtecClass_'+str(object_class) +'_checkpoint.pth'
target_layer = 'encoder.layer2.1.conv1'

In [ ]:
run -t  code/train_expVAE.py --dataset=mvtec_ad --model=resnet18_3 --batch_size=8 --one_class=$object_class

### To test the self-trained model:

**Info:** to find the best IoU score set the flag: --iou True

In [ ]:
run code/test_expVAE.py --dataset=mvtec_ad --model=resnet18_3 --batch_size=2 --model_path=$self_trained_model --one_class=$object_class --target_layer=$target_layer

### To load pretrained models:

In [ ]:
run -t code/test_expVAE.py --dataset=mvtec_ad --model=resnet18_3 --batch_size=2 --model_path=$pre_trained_model --one_class=$object_class --target_layer=$target_layer

In [ ]:
run -t code/show_results.py --dataset=mvtec_ad --model=resnet18_3 --one_class=$object_class

## dSprites Latent Space Disentanglement
Make sure you are in the Latent_Space_Disentanglement directory

In [ ]:
cd ../Latent_Space_Disentanglement/

If you have not yet downloaded the dSprites dataset, uncomment the following cell and run it.

In [ ]:
# !sh scripts/prepare_data.sh dsprites

### Visualizing attention maps
First we reproduce the attention maps of the two highest response latent dimensions for the baseline FactorVAE and the best performing AD-FactorVAE using $\lambda=40$ and the first convolutional layer. Press any key to close the image. (Using the ```--help``` flag you can find more options)

In [ ]:
!python visualizer.py --name FactorVAE --target_layer 0 --cuda --sample_count 10

In [ ]:
!python visualizer.py --name AD-FactorVAE --target_layer 0 --cuda --sample_count 10

### Plotting the results
Next we plot the results for all the models used, corresponding to 3a in the report. Using the ```--help``` flag you can find more options, for example ```--all_plots``` will show more information about the loss progression.

In [ ]:
%run plotter.py --names 'gamma40 lambda1_gamma40 lambda20_gamma40 lambda40_gamma40 conv3_lambda1_gamma40'